In [1]:
import torch, transformers, accelerate, numpy, os

print("Torch:", torch.__version__)
print("Transformers:", transformers.__version__)
print("Accelerate:", accelerate.__version__)
print("Numpy:", numpy.__version__)



Torch: 2.6.0+cu124
Transformers: 4.53.2
Accelerate: 1.9.0
Numpy: 2.0.2


In [2]:
!pip install evaluate

In [3]:
pip install rouge_score

In [4]:
!pip install evaluate sacrebleu


In [5]:
import os
os.environ["WANDB_DISABLED"] = "true"

import pandas as pd
import numpy as np
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq, EarlyStoppingCallback
from datasets import Dataset
import evaluate
from sklearn.model_selection import train_test_split
import torch

raw_data = pd.read_csv("hf://datasets/HeshamHaroon/ArzEn-MultiGenre/dataset.csv", nrows=20000)
print("first 10 rows:")
print(raw_data[["ENG", "EGY"]].head(10))

data = raw_data.dropna(subset=["ENG", "EGY"])
data = data.drop_duplicates()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


first 10 rows:
                                                 ENG  \
0                                           Already?   
1                         Sorry to keep you waiting.   
2                                        No problem.   
3                               The system was down.   
4  Well, we closed your joint account. We added H...   
5                     All that with the system down?   
6               You've always worked miracles, Soha.   
7                             No, we got used to it.   
8  You're not the first to divorce, but you were ...   
9  I know Ola doesn't work. She dedicated her lif...   

                                                 EGY  
0                                             ‫لحق؟‬  
1                            ‫معلش يا جماعة أخرتكم.‬  
2                                    ‫لا، ولا يهمك.‬  
3                                ‫بس الsystem down.‬  
4  ‫طيب. خلاص إحنا كدا قفلنا الjoint account ‫زود...  
5                            ‫كل ده وا

In [6]:
def is_invalid_text(text):
    text = str(text).strip()
    return bool(re.fullmatch(r"[\d\s\W]*", text))

data = data[~data["ENG"].apply(is_invalid_text)]
data = data[~data["EGY"].apply(is_invalid_text)]
data = data[data["ENG"].str.len() > 1]
data = data[data["EGY"].str.len() > 1]
data = data.reset_index(drop=True)

In [7]:
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r"\s+", " ", text)  # إزالة المسافات الزايدة فقط
    text = text.strip()
    return text

data["ENG"] = data["ENG"].apply(clean_text)
data["EGY"] = data["EGY"].apply(clean_text)

data.head(10)

,EGY,ENG
0,‫لحق؟‬,already?
1,‫معلش يا جماعة أخرتكم.‬,sorry to keep you waiting.
2,‫لا، ولا يهمك.‬,no problem.
3,‫بس الsystem down.‬,the system was down.
4,‫طيب. خلاص إحنا كدا قفلنا الjoint account ‫زود...,"well, we closed your joint account. we added h..."
5,‫كل ده والsystem down؟‬,all that with the system down?
6,‫لا إنتي بتصنعي المعجزات يا سها طول عمرك.‬,"you've always worked miracles, soha."
7,‫لا خالص. إحنا بس اتعودنا.‬,"no, we got used to it."
8,‫من كتر حالات الطلاق ومشاكل النفقة.‬ ‫بس إنتوا...,"you're not the first to divorce, but you were ..."
9,‫أنا بس عشان عارف إن علا مش بتشتغل‬ ‫وكانت مدي...,i know ola doesn't work. she dedicated her lif...


In [8]:
print("Sample Data after cleaning (first 10 rows):")
print(data[["ENG", "EGY"]].head(10))
print("Number of rows after cleaning:", len(data))

# التأكد من إن الداتاسيت مش فارغة
if len(data) == 0:
    raise ValueError("Field")

train_data, test_data = train_test_split(data, test_size=0.2, random_state=42, shuffle=True)
print("Train size:", len(train_data))
print("Test size:", len(test_data))

Sample Data after cleaning (first 10 rows):
                                                 ENG  \
0                                           already?   
1                         sorry to keep you waiting.   
2                                        no problem.   
3                               the system was down.   
4  well, we closed your joint account. we added h...   
5                     all that with the system down?   
6               you've always worked miracles, soha.   
7                             no, we got used to it.   
8  you're not the first to divorce, but you were ...   
9  i know ola doesn't work. she dedicated her lif...   

                                                 EGY  
0                                             ‫لحق؟‬  
1                            ‫معلش يا جماعة أخرتكم.‬  
2                                    ‫لا، ولا يهمك.‬  
3                                ‫بس الsystem down.‬  
4  ‫طيب. خلاص إحنا كدا قفلنا الjoint account ‫زود...  
5        

In [9]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import Dataset
import torch

model_name = "Helsinki-NLP/opus-mt-en-ar"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

source.spm:   0%|          | 0.00/801k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/917k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/308M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [10]:
train_dataset = Dataset.from_pandas(train_data)
test_dataset = Dataset.from_pandas(test_data)

In [11]:
prefix = "translate English to Egyptian: "
def preprocess_function(batch):
    inputs = [prefix + text for text in batch["ENG"]]
    targets = batch["EGY"]
    print("Sample Inputs (before tokenization):", inputs[:5])
    print("Sample Targets (before tokenization):", targets[:5])
    model_inputs = tokenizer(inputs, max_length=24, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=16, truncation=True, padding="max_length")
    print("Sample Input IDs:", model_inputs["input_ids"][:5])
    print("Sample Label IDs:", labels["input_ids"][:5])
    decoded_inputs = tokenizer.batch_decode(model_inputs["input_ids"], skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels["input_ids"], skip_special_tokens=True)
    print("Sample Inputs (decoded):", decoded_inputs[:5])
    print("Sample Targets (decoded):", decoded_labels[:5])
    for i, input_ids in enumerate(model_inputs["input_ids"]):
        if all(id in [tokenizer.pad_token_id, tokenizer.eos_token_id] for id in input_ids):
            print(f"Warning: Input {i} is all padding tokens!")
    for i, label_ids in enumerate(labels["input_ids"]):
        if all(id in [tokenizer.pad_token_id, tokenizer.eos_token_id] for id in label_ids):
            print(f"Warning: Label {i} is all padding tokens!")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Make Tokenization
tokenized_train = train_dataset.map(preprocess_function, batched=True, remove_columns=["ENG", "EGY"])
tokenized_test = test_dataset.map(preprocess_function, batched=True, remove_columns=["ENG", "EGY"])

#  BLEU & ROUGE
bleu = evaluate.load("sacrebleu")
rouge = evaluate.load("rouge")



Map:   0%|          | 0/13925 [00:00<?, ? examples/s]

Sample Inputs (before tokenization): ['translate English to Egyptian: he makes you do it?', 'translate English to Egyptian: i mean, it was fine.', "translate English to Egyptian: she's good, he thought.", "translate English to Egyptian: as if they're homemade, made with love and care.", "translate English to Egyptian: i'll stay here."]
Sample Targets (before tokenization): ['هو اللي مشغلك', 'بصي هو وسطي.', 'فكر، دي مركب بنت حلال.', '\u202bكأنه معمول في البيت كدا بعناية وحب.\u202c', 'أنا قاعد']


/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3951: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Sample Input IDs: [[17958, 3617, 9, 16930, 50, 155, 2944, 30, 144, 44, 32, 0, 62801, 62801, 62801, 62801, 62801, 62801, 62801, 62801, 62801, 62801, 62801, 62801], [17958, 3617, 9, 16930, 50, 2201, 874, 5, 44, 53, 2364, 2, 0, 62801, 62801, 62801, 62801, 62801, 62801, 62801, 62801, 62801, 62801, 62801], [17958, 3617, 9, 16930, 50, 397, 16, 18, 465, 5, 155, 1399, 2, 0, 62801, 62801, 62801, 62801, 62801, 62801, 62801, 62801, 62801, 62801], [17958, 3617, 9, 16930, 50, 42, 252, 170, 16, 120, 1229, 21579, 5, 229, 34, 1195, 8, 995, 2, 0, 62801, 62801, 62801, 62801], [17958, 3617, 9, 16930, 50, 2201, 16, 261, 2631, 259, 2, 0, 62801, 62801, 62801, 62801, 62801, 62801, 62801, 62801, 62801, 62801, 62801, 62801]]
Sample Label IDs: [[145, 19046, 43520, 93, 0, 62801, 62801, 62801, 62801, 62801, 62801, 62801, 62801, 62801, 62801, 62801], [8229, 47, 145, 4873, 47, 2, 0, 62801, 62801, 62801, 62801, 62801, 62801, 62801, 62801, 62801], [15952, 7, 2483, 27147, 20151, 1812, 288, 2, 0, 62801, 62801, 62801, 6

Map:   0%|          | 0/3482 [00:00<?, ? examples/s]

Sample Inputs (before tokenization): ["translate English to Egyptian: it's not a joke.", 'translate English to Egyptian: the one thing that will never change is how i feel about you and your brother.', "translate English to Egyptian: some details are different, but i'm the same on the inside.", "translate English to Egyptian: you should know you're not allowed to open someone else's fridge.", 'translate English to Egyptian: to help me and to see this.']
Sample Targets (before tokenization): ['أصله مش هزار على فكرة.', '\u202bوأكتر حاجة بصراحة مش عايزها تتغير\u202c \u202bهي علاقتي بيكي إنتي وأخوكي.\u202c', '\u202bيمكن غيرت شوية تفاصيل\u202c \u202bفي حياتي، لكن أنا لا.\u202c', '\u202bحبيبتي هو محدش قالك\u202c \u202bإنك ميصحش تفتحي التلاجة في بيت حد؟\u202c', 'كان ساعدني ويشوف اللي بيحصل ده .']
Sample Input IDs: [[17958, 3617, 9, 16930, 50, 44, 16, 18, 66, 15, 15408, 2, 0, 62801, 62801, 62801, 62801, 62801, 62801, 62801, 62801, 62801, 62801, 62801], [17958, 3617, 9, 16930, 50, 3, 171, 1101,

In [12]:

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.array(predictions, dtype=np.int32)
    labels = np.array(labels, dtype=np.int32)

    print("Sample Predictions (token IDs):", predictions[:5])
    print("Sample Labels (token IDs):", labels[:5])

    labels = np.where(labels == -100, tokenizer.pad_token_id, labels)
    predictions = np.clip(predictions, 0, tokenizer.vocab_size - 1)
    labels = np.clip(labels, 0, tokenizer.vocab_size - 1)

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [label.strip() for label in decoded_labels]

    print("Sample Predictions (decoded):", decoded_preds[:5])
    print("Sample References (decoded):", decoded_labels[:5])

    result = bleu.compute(predictions=decoded_preds, references=[[label] for label in decoded_labels], smooth_method="floor")
    rouge_result = rouge.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"], "rouge": rouge_result}

# إعدادات التدريب
training_args = Seq2SeqTrainingArguments(
    output_dir="./t5-egy-model",
    run_name="t5-egyptian-translation",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=30,
    predict_with_generate=True,
    logging_dir="./logs",
    logging_steps=100,
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="bleu",
    greater_is_better=True,
    report_to="none",
)

In [13]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]  # زيادة patience
)



/tmp/ipython-input-13-852986752.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [14]:
trainer.train()

Epoch,Training Loss,Validation Loss,Bleu,Rouge
1,1.944800,1.882318,6.305862,"{'rouge1': 0.0049205437488033694, 'rouge2': 0.0005743825387708214, 'rougeL': 0.004977982002680452, 'rougeLsum': 0.004872678537239135}"
2,1.565100,1.718546,7.588852,"{'rouge1': 0.007452613440551408, 'rouge2': 0.0017231476163124641, 'rougeL': 0.00759142255408769, 'rougeLsum': 0.007457399961707831}"
3,1.390900,1.633743,8.404473,"{'rouge1': 0.010913268236645606, 'rouge2': 0.002441125789775991, 'rougeL': 0.010865403025081372, 'rougeLsum': 0.010817537813517134}"
4,1.108700,1.582766,9.740692,"{'rouge1': 0.012066819835343672, 'rouge2': 0.002895845299636224, 'rougeL': 0.012085965919969365, 'rougeLsum': 0.011985448975684473}"
5,1.042100,1.550808,10.651772,"{'rouge1': 0.012645988895270915, 'rouge2': 0.003242868083476929, 'rougeL': 0.01265077541642734, 'rougeLsum': 0.01262684281064522}"
6,0.920700,1.538509,11.004448,"{'rouge1': 0.013430978364924373, 'rouge2': 0.0036138234730997507, 'rougeL': 0.0134166188014551, 'rougeLsum': 0.013363967068734443}"
7,0.834100,1.513042,11.715430,"{'rouge1': 0.015000957304231283, 'rouge2': 0.004810453762205629, 'rougeL': 0.015163699023549684, 'rougeLsum': 0.015029676431169824}"
8,0.720300,1.503746,12.694242,"{'rouge1': 0.01646768414430677, 'rouge2': 0.005238847405705533, 'rougeL': 0.016521703454500693, 'rougeLsum': 0.0164382812286316}"
9,0.597600,1.503702,12.882591,"{'rouge1': 0.016819835343672217, 'rouge2': 0.005313038483630098, 'rougeL': 0.016805475780202944, 'rougeLsum': 0.01673846448401302}"
10,0.555400,1.505638,13.762513,"{'rouge1': 0.017567216432810916, 'rouge2': 0.005973578403216543, 'rougeL': 0.017678673996881916, 'rougeLsum': 0.01757952463007029}"


Sample Predictions (token IDs): [[62801  5235   134 ... 62801 62801 62801]
 [62801  3484  2841 ... 62801 62801 62801]
 [62801   320 10067 ... 62801 62801 62801]
 [62801   253 27333 ... 62801 62801 62801]
 [62801  4941 19037 ... 62801 62801 62801]]
Sample Labels (token IDs): [[ 5087    38  5235 ... 62801 62801 62801]
 [ 2199 17508    93 ... 62801 62801 62801]
 [ 2199 20407   109 ... 62801 62801 62801]
 [ 2199 33121   647 ... 62801 62801 62801]
 [  131 21629   380 ... 62801 62801 62801]]
Sample Predictions (decoded): ['مش مزحه.', 'الشيء الوحيد اللي مش هتغير أبدا هو شعوري تجاهك وتجاه أخوك.', 'بعض التفاصيل مختلفة، بس أنا كمان في الداخل.', 'أنت عارف إنك مش مسموح لك تفتح ثلاجة غيرك.', 'علشان تساعدني وأشوف كدا.']
Sample References (decoded): ['أصله مش هزار على فكرة.', '\u202bوأكتر حاجة بصراحة مش عايزها تتغير\u202c \u202bهي', '\u202bيمكن غيرت شوية تفاصيل\u202c \u202bفي حياتي، لكن أنا لا', '\u202bحبيبتي هو محدش قالك\u202c \u202bإنك ميصحش تفتح', 'كان ساعدني ويشوف اللي بيحصل ده .']


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3685: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[62801]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


Sample Predictions (token IDs): [[62801  2483  5235 ... 62801 62801 62801]
 [62801  3484  2841 ... 62801 62801 62801]
 [62801  2199   288 ... 62801 62801 62801]
 [62801 37158 20836 ... 62801 62801 62801]
 [62801 12564   239 ... 62801 62801 62801]]
Sample Labels (token IDs): [[ 5087    38  5235 ... 62801 62801 62801]
 [ 2199 17508    93 ... 62801 62801 62801]
 [ 2199 20407   109 ... 62801 62801 62801]
 [ 2199 33121   647 ... 62801 62801 62801]
 [  131 21629   380 ... 62801 62801 62801]]
Sample Predictions (decoded): ['دي مش نكتة.', 'الشيء الوحيد اللي مش هتغير أبدا هو شعوري تجاهك وتجاه أخوك', '\u202bالتفاصيل مختلفة،\u202c \u202bبس أنا كمان.\u202c', 'لازم تعرفي مش مسموح لك تفتحي ثلاجة حد تاني.', 'عشان تساعدني وأشوفها.']
Sample References (decoded): ['أصله مش هزار على فكرة.', '\u202bوأكتر حاجة بصراحة مش عايزها تتغير\u202c \u202bهي', '\u202bيمكن غيرت شوية تفاصيل\u202c \u202bفي حياتي، لكن أنا لا', '\u202bحبيبتي هو محدش قالك\u202c \u202bإنك ميصحش تفتح', 'كان ساعدني ويشوف اللي بيحصل ده .']
Sam

There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.encoder.embed_positions.weight', 'model.decoder.embed_tokens.weight', 'model.decoder.embed_positions.weight', 'lm_head.weight'].


TrainOutput(global_step=52230, training_loss=0.5139838579717596, metrics={'train_runtime': 7199.0161, 'train_samples_per_second': 58.029, 'train_steps_per_second': 7.255, 'total_flos': 2655195365376000.0, 'train_loss': 0.5139838579717596, 'epoch': 30.0})

In [16]:
def translate_to_egyptian(sentence, max_input_length=64, max_output_length=64):
    model.eval()
    input_text = f"translate English to Egyptian: {sentence}"

    # Tokenization
    inputs = tokenizer(input_text, return_tensors="pt", max_length=max_input_length, truncation=True).to(model.device)

    # Generate translation
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_output_length,
            num_beams=4,
            early_stopping=True
        )

    # Decode output
    translated = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return translated


In [25]:
# قائمة الجمل الإنجليزية
examples = [
    "How are you doing today?",
    "dont talk any more ",
    "I love Egyptian food.",
    "This place is beautiful.",
    "Are you coming with us?",
    "What are you doing tomorrow?",
    "I'm tired and need to rest.",
    "Let's watch a movie.",
    "Can I help you?",
    "Don't be late!"
]

# ترجمة كل جملة
for i, sentence in enumerate(examples, 1):
    translation = translate_to_egyptian(sentence)
    print(f"{i}. EN: {sentence}")
    print(f"   EG: {translation}")
    print()


1. EN: How are you doing today?
   EG: ازيكو النهاردة؟

2. EN: dont talk any more 
   EG: متتكلميش!

3. EN: I love Egyptian food.
   EG: أنا بحب الأكل المصري

4. EN: This place is beautiful.
   EG: ده المكان جميل اوي.

5. EN: Are you coming with us?
   EG: هتيجي معانا؟

6. EN: What are you doing tomorrow?
   EG: هتمفي إيه بكرة؟

7. EN: I'm tired and need to rest.
   EG: أنا تعبان ومحتاج أرتاح.

8. EN: Let's watch a movie.
   EG: هنروح نشوف فيلما

9. EN: Can I help you?
   EG: طب ممكن أساعدك؟

10. EN: Don't be late!
   EG: ‫متتأخرش!‬

